# Прогон тяжелых экспериментов (Гипотеза 1 и 2)

Ноутбук запускает `python3 main.py` с разными конфигами и сохраняет отдельные CSV-файлы результатов.

In [ ]:
from pathlib import Path
import shutil
import subprocess
from datetime import datetime

REPO = Path('/Users/mihailledenev/PycharmProjects/AI_360_project')
CONFIG_DIR = REPO / 'config'
RESULTS_DIR = REPO / 'results'
SCENARIOS_PATH = CONFIG_DIR / 'scenarios.json'
MAIN = REPO / 'main.py'

assert REPO.exists(), REPO
assert MAIN.exists(), MAIN
assert (CONFIG_DIR / 'hypothesis_price.json').exists()
assert (CONFIG_DIR / 'hypothesis_liquidity.json').exists()
print('Workspace:', REPO)

## Настройки запуска
- `CLEAR_BEFORE_RUN=True` удалит текущий `results/summary.csv` и png в `results/plots/` перед **каждым** запуском.
- `RUN_PRICE` и `RUN_LIQUIDITY` позволяют запускать сценарии выборочно.

In [ ]:
RUN_PRICE = True
RUN_LIQUIDITY = True
CLEAR_BEFORE_RUN = False
STAMP = datetime.now().strftime('%Y%m%d_%H%M%S')

print('RUN_PRICE=', RUN_PRICE)
print('RUN_LIQUIDITY=', RUN_LIQUIDITY)
print('CLEAR_BEFORE_RUN=', CLEAR_BEFORE_RUN)
print('STAMP=', STAMP)

In [ ]:
def clear_results():
    summary = RESULTS_DIR / 'summary.csv'
    if summary.exists():
        summary.unlink()
    plots_dir = RESULTS_DIR / 'plots'
    if plots_dir.exists():
        for p in plots_dir.glob('*.png'):
            p.unlink()

def run_with_config(config_file: str, output_csv_name: str):
    src = CONFIG_DIR / config_file
    assert src.exists(), src

    if CLEAR_BEFORE_RUN:
        clear_results()

    shutil.copy2(src, SCENARIOS_PATH)
    print(f'\n=== Running: {config_file} ===')
    cmd = ['python3', str(MAIN)]
    proc = subprocess.run(cmd, cwd=str(REPO), check=False)
    if proc.returncode != 0:
        raise RuntimeError(f'Run failed for {config_file}, code={proc.returncode}')

    summary = RESULTS_DIR / 'summary.csv'
    if not summary.exists():
        raise FileNotFoundError(summary)

    out = RESULTS_DIR / output_csv_name
    shutil.copy2(summary, out)
    print(f'Saved: {out}')
    return out


## Запуск гипотезы 1 (ценовые шоки)

In [ ]:
price_out = None
if RUN_PRICE:
    price_out = run_with_config(
        config_file='hypothesis_price.json',
        output_csv_name=f'summary_hypothesis_price_{STAMP}.csv'
    )
price_out

## Запуск гипотезы 2 (шоки ликвидности)

In [ ]:
liq_out = None
if RUN_LIQUIDITY:
    liq_out = run_with_config(
        config_file='hypothesis_liquidity.json',
        output_csv_name=f'summary_hypothesis_liquidity_{STAMP}.csv'
    )
liq_out

## Быстрая проверка файлов

In [ ]:
for p in sorted(RESULTS_DIR.glob(f'*{STAMP}*.csv')):
    print(p.name)
